In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
from torch import optim
import torchvision
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import re
import os
from collections import Counter
import csv

import nltk
from nltk.tokenize import word_tokenize
from nltk import PorterStemmer
from nltk import WordNetLemmatizer
from torch.utils.data import TensorDataset, DataLoader
from nltk.corpus import stopwords

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split

импорт нужных библиотек и считывание датасета из файла

In [17]:
from google.colab import files
data_to_load = files.upload()

Saving reviews.csv to reviews (1).csv


In [20]:
import io
df = pd.read_csv(io.BytesIO(data_to_load['reviews (1).csv']))
df

,review,star
0,"Банк столько раз меня обманывал, что жду не до...",1
1,К клиентам никакого уважения- не зависит от ка...,1
2,Добрый день!,1
3,"и маркетплейсах, а также в продуктовых магазин...",1
4,27 октября в отделении банка было создано обра...,1
...,...,...
2963,"Здравствуйте,хочется поблагодарить работника о...",5
2964,NaN,5
2965,Когда-то давным-давно я пользовалась картой Ал...,5
2966,Хочется похвалить сотрудников онлайн-чата и со...,5


удаляем пустые строки из датасета

In [21]:
df.dropna(inplace =True)
df

,review,star
0,"Банк столько раз меня обманывал, что жду не до...",1
1,К клиентам никакого уважения- не зависит от ка...,1
2,Добрый день!,1
3,"и маркетплейсах, а также в продуктовых магазин...",1
4,27 октября в отделении банка было создано обра...,1
...,...,...
2962,"В сентябре я оформил карту Альфа-Банка, тут же...",5
2963,"Здравствуйте,хочется поблагодарить работника о...",5
2965,Когда-то давным-давно я пользовалась картой Ал...,5
2966,Хочется похвалить сотрудников онлайн-чата и со...,5


обработка текста отзыва

In [23]:
remove_non_alphabets = lambda x: re.sub(r'[^\w\s]', ' ', x, flags=re.UNICODE)

tokenize = lambda x: word_tokenize(x)

ps = PorterStemmer()
stem = lambda w: [ps.stem(x) for x in w]

lemmatizer = WordNetLemmatizer()
leammtizer = lambda x: [lemmatizer.lemmatize(word) for word in x]

counter = lambda x: Counter(x)
sort_words = lambda x: list((dict(sorted(x.items(), key=lambda item: item[1], reverse=True)[:500])).keys())

In [15]:
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [24]:
print('Start processing')
df['review'] = df['review'].apply(remove_non_alphabets)
print('removed non alphabets')
df['review'] = df['review'].apply(tokenize)
print('tokenized')
df['review'] = df['review'].apply(stem)
print('stemmed')
df['review'] = df['review'].apply(leammtizer)
print('lemmatized')
df['review'] = df['review'].apply(counter)
print('counted')
df['review'] = df['review'].apply(sort_words)
print('sorted')
df['review'] = df['review'].apply(lambda x: ' '.join(x))
print('End of processing')
df

Start processing
removed non alphabets
tokenized
stemmed
lemmatized
counted
sorted
End of processing


,review,star
0,банк столько раз меня обманывал что жду не дож...,1
1,не от к клиентам никакого уважения зависит кат...,1
2,добрый день,1
3,и маркетплейсах а также в продуктовых магазина...,1
4,и в обращение предложений по кредиту кредитным...,1
...,...,...
2962,и за бонус в сентябре я оформил карту альфа ба...,5
2963,карты все здравствуйте хочется поблагодарить р...,5
2965,не и на все я но карту это мне тимур так что а...,5
2966,в и банка документов сотрудников чата обратилс...,5


векторизация:

In [25]:
russian_stopwords = stopwords.words('russian')

In [26]:
max_words = 500
cv = CountVectorizer(max_features=max_words, stop_words=russian_stopwords)
sparse_matrix = cv.fit_transform(df['review']).toarray()

In [27]:
sparse_matrix.shape

(2912, 500)